## Key Topics Extraction and Contextual Sentiment of Hotel Reviews

### 1. Preprocess

In [31]:
import os
from os import listdir
import pandas as pd
import plotly.graph_objs as go
df = pd.read_csv('/Users/lancy/Desktop/Topics-Extraction-Hotel-Reviews/Nlp project/hotel-reviews/Datafiniti_Hotel_Reviews_Jun19.csv', header=0)

df.drop(columns=['dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'keys', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username', 'sourceURLs', 'websites'], inplace = True)
df = df.rename(index=str, columns={'reviews.rating':'rating', 'reviews.sourceURLs': 'reviews_sourceURLs', 'reviews.text':'reviews_text', 'reviews.title': 'reviews_title'})

grouped = df.groupby('name')
df1 = grouped.get_group("Best Western Carmel's Town House Lodge")

df1 = df1[['reviews_text','name','city']]


In [32]:
df1

,reviews_text,name,city
1,We stayed in the king suite with the separatio...,Best Western Carmel's Town House Lodge,Carmel by the Sea
2,"Parking was horrible, somebody ran into my ren...",Best Western Carmel's Town House Lodge,Carmel by the Sea
3,Not cheap but excellent location. Price is som...,Best Western Carmel's Town House Lodge,Carmel by the Sea
4,If you get the room that they advertised on th...,Best Western Carmel's Town House Lodge,Carmel by the Sea


In [33]:

f= open("hotel_review.txt","w+")
for i in range(df.shape[0]):
    f.write(df.reviews_text[i])
    f.write("\n")
f.close()

with open("hotel_review.txt") as fp: 
    reviews = []
    Lines = fp.readlines() 
    for line in Lines[:5]: 
        reviews.append(line)

In [34]:
reviews

['This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before.\n',
 "We stayed in the king suite with the separation between the bedroom and the living space. The sofa bed wasn't very good I had back discomfort by the day we left on our three night stay. The room is clean, and the king bed very comfortable. This hotel is located within walking distance to most places you will want to... More\n",
 "Parking was horrible, somebody ran into my rental car while staying there. I didn't get to try the breakfast, I was there for business so the restaurant opened to late for the business world to enjoy, I had to asked for coffee for my room, And the items in the vending machine were stale.\n",
 'Not cheap but excellent location. Price is somewhat standard for not hacing reservations. But room was nice and clean. They offer good continental bre

In [35]:
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(reviews):
    stop_free = " ".join([i for i in reviews.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

reviews_clean = [clean(reviews).split() for reviews in reviews]    

dictionary = corpora.Dictionary(reviews_clean)

In [36]:
reviews_clean

[['hotel',
  'nice',
  'quiet',
  'know',
  'train',
  'track',
  'near',
  'by',
  'train',
  'passed',
  'stay',
  'best',
  'western',
  'changed',
  'hotel',
  'classification',
  'plus',
  'category',
  'before'],
 ['stayed',
  'king',
  'suite',
  'separation',
  'bedroom',
  'living',
  'space',
  'sofa',
  'bed',
  'good',
  'back',
  'discomfort',
  'day',
  'left',
  'three',
  'night',
  'stay',
  'room',
  'clean',
  'king',
  'bed',
  'comfortable',
  'hotel',
  'located',
  'within',
  'walking',
  'distance',
  'place',
  'want',
  'to'],
 ['parking',
  'horrible',
  'somebody',
  'ran',
  'rental',
  'car',
  'staying',
  'there',
  'get',
  'try',
  'breakfast',
  'business',
  'restaurant',
  'opened',
  'late',
  'business',
  'world',
  'enjoy',
  'asked',
  'coffee',
  'room',
  'item',
  'vending',
  'machine',
  'stale'],
 ['cheap',
  'excellent',
  'location',
  'price',
  'somewhat',
  'standard',
  'hacing',
  'reservation',
  'room',
  'nice',
  'clean',
  'o

In [37]:
dictionary

In [38]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

reviews_term_matrix = [dictionary.doc2bow(reviews) for reviews in reviews_clean]

In [42]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(reviews_term_matrix, num_topics=3, id2word = dictionary, passes=5)

In [46]:
print(ldamodel.print_topics(num_topics=1, num_words=6))

[(1, '0.030*"hotel" + 0.030*"train" + 0.029*"business" + 0.017*"plus" + 0.017*"near" + 0.017*"best"')]
